## Homework: Agents
In this homework, we will learn more about function calling, and we will also explore MCP - model-context protocol.

### Preparation

First, we'll define a function that we will use when building our agent.

It will generate fake weather data:

In [9]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description
We want to use it as a tool for our agent, so we need to describe it

How should the description for this function look like? Fill in missing parts

In [2]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",  # TODO1
    "description": "Retrieves the temperature for a specified city.",  # TODO2
    "parameters": {
        "type": "object",
        "properties": {
            "city": {  # TODO3
                "type": "string",
                "description": "The name of the city to get weather information for."  # TODO4
            }
        },
        "required": ["city"],  # TODO5
        "additionalProperties": False
    }
}


### Testing it (Optional)
If you have OpenAI API Key (or alternative provider), let's test it.

A question could be "What's the weather like in Germany?"

Experiment with different system prompts to have better answers from the system.

You can use chat_assistant.py or implement everything yourself

```shell
wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
```


### Q2. Adding another tool
Let's add another tool - a function that can add weather data to our database:



In [10]:

def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

Now let's write a description for it.

What did you write?

Optionally, you can test it after adding this function.


In [11]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Sets the temperature for a specified city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to set the weather for."
            },
            "temp": {
                "type": "number",
                "description": "The temperature to set for the city."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}


In [12]:
import chat_assistant



tools = chat_assistant.Tools()
tools.add_tool(set_weather, set_weather_tool)

In [13]:
tools.get_tools()

[{'type': 'function',
  'name': 'set_weather',
  'description': 'Sets the temperature for a specified city.',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The name of the city to set the weather for.'},
    'temp': {'type': 'number',
     'description': 'The temperature to set for the city.'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

### MCP

---

MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:

A tool can export a list of functions it has
When we include the tool to our Agent, we just need to include the link to the MCP server

## Q3. Install FastMCP
Let's install a library for MCP - FastMCP:



What's the version of FastMCP you installed?



In [6]:
pip show fastmcp

Name: fastmcp
Version: 2.10.5
Summary: The fast, Pythonic way to build MCP servers and clients.
Home-page: https://gofastmcp.com
Author: Jeremiah Lowin
Author-email: 
License-Expression: Apache-2.0
Location: /home/horus/anaconda3/envs/mcpenv/lib/python3.11/site-packages
Requires: authlib, cyclopts, exceptiongroup, httpx, mcp, openapi-pydantic, pydantic, pyperclip, python-dotenv, rich
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Q4. Simple MCP Server
---
A simple MCP server from the documentation looks like that:

In our case, we need to write docstrings for our functions.

Let's ask ChatGPT for help:

In [ ]:
def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [8]:
!python fastmcp_server.py



╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                             │
│    📦 Transport:       STDIO                                               │
│                                                  

## Q5. Protocol

There are different ways to communicate with an MCP server. Ours is currently running using standart input/output, which means that the client write something to stdin and read the answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:



First, we send an initialization request -- this way, we register our client with the server:

```json
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```

We should get back something like that, which is an aknowledgement of the request:
```json
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
```

Next, we reply back, confirming the initialization:

```json
{"jsonrpc": "2.0", "method": "notifications/initialized"}
```

We don't expect to get anything in response
Now we can ask for a list of available methods:

```json
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```

Let's ask the temperature in Berlin:
```json
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "<TODO>", "arguments": {<TODO>}}}
```

What did you get in response?

```json
{
  "jsonrpc": "2.0",
  "id": 3,
  "method": "tools/call",
  "params": {
    "name": "get_weather",
    "arguments": {
      "city": "Berlin"
    }
  }
}
```

In [14]:
from openai import OpenAI
import os
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key= os.getenv("LLM_API_KEY"),
  )


developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

## Q6. Client



We typically don't interact with the server by copy-pasting commands in the terminal.

In practice, we use an MCP Client. Let's implement it.

FastMCP also supports MCP clients:

```python
from fastmcp import Client

async def main():
    async with Client(<TODO>) as mcp_client:
        # TODO
```

Use the client to get the list of available tools of our script. How does the result look like?

If you're running this code in Jupyter, you need to pass an instance of MCP server to the Client:


```python
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # ....

```

If you run it in a script, you will need to use asyncio:

```python
import asyncio

async def main():
    async with Client("weather_server.py") as mcp_client:
        # ...

if __name__ == "__main__":
    test = asyncio.run(main())
```

Copy the output with the available tools when filling in the homework form.